<h2> BPE Algorithm </h2>

In [ ]:
from collections import Counter, defaultdict

class BPE():

    def __init__(self, corpus, vocab_size):
        self.corpus = corpus
        self.vocab_size = vocab_size
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.word_freqs = defaultdict(int)
        self.splits = {}
        self.merges = {}
    
    
    def train(self):
        for text in self.corpus:
            #words_with_offsets = self.tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
            words_without_offsets = text.split()   
            new_words = [word for word in words_without_offsets]
            for word in new_words:
                self.word_freqs[word] += 1

        alphabet = []
        for word in self.word_freqs.keys():
            for letter in word:
                if letter not in alphabet:
                    alphabet.append(letter)
        alphabet.sort()

        vocab = ["</w>"] + alphabet.copy()

        self.splits = {word: [c for c in word] for word in self.word_freqs.keys()}

        for i in range(vocab_size):
            pair_freqs = self.compute_pair_freqs()

            best_pair = ""
            max_freq = None
            for pair, freq in pair_freqs.items():
                if max_freq is None or max_freq < freq:
                    best_pair = pair
                    max_freq = freq

            self.splits = self.merge_pair(*best_pair)
            self.merges[best_pair] = best_pair[0] + best_pair[1]
            vocab.append(best_pair[0] + best_pair[1])
        return vocab


    def compute_pair_freqs(self):

        pair_freqs = defaultdict(int)
        for word, freq in self.word_freqs.items():
            split = self.splits[word]
            if len(split) == 1:
                continue
            for i in range(len(split) - 1):
                pair = (split[i], split[i + 1])
                pair_freqs[pair] += freq
        return pair_freqs


    def merge_pair(self, a, b):

        for word in self.word_freqs:
            split = self.splits[word]
            if len(split) == 1:
                continue
            i = 0
            while i < len(split) - 1:
                if split[i] == a and split[i + 1] == b:
                    split = split[:i] + [a + b] + split[i + 2 :]
                else:
                    i += 1
            self.splits[word] = split
        return self.splits
    

    def tokenize(self, text):
        #pre_tokenize_result = self.tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
        pre_tokenize_result = text.split()        
        #pre_tokenized_text = [word for word, offset in pre_tokenize_result]
        pre_tokenized_text = [word for word in pre_tokenize_result]

        splits_text = [[l for l in word] for word in pre_tokenized_text]

        for pair, merge in self.merges.items():
            for idx, split in enumerate(splits_text):
                i = 0
                while i < len(split) - 1:
                    if split[i] == pair[0] and split[i + 1] == pair[1]:
                        split = split[:i] + [merge] + split[i + 2 :]
                    else:
                        i += 1
                splits_text[idx] = split
        result = sum(splits_text, [])
        return result

<h2> Wordpiece  Algorithm </h2>

In [ ]:
class WordPiece():

    def __init__(self, corpus, vocab_size):
        self.corpus = corpus
        self.vocab_size = vocab_size
        # pre-tokenize the corpus into words, BERT pre-tokenizer is used here
        # self.tokenizer = bertTokrizer
        self.word_freqs = defaultdict(int)
        self.splits = {}
        self.merges = {}
        self.word = {}
        self.vocab = None
    
    
    def train(self):
        for text in self.corpus:
            # words_with_offsets = self.tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
            # new_words = [word for word, offset in words_with_offsets]
            words_without_offsets = text.split()
            new_words = [word for word in words_without_offsets]
            for word in new_words:
                self.word_freqs[word] += 1

        alphabet = []
        for word in self.word_freqs.keys():
            if word[0] not in alphabet:
                alphabet.append(word[0])
            for letter in word[1:]:
                if f"##{letter}" not in alphabet:
                    alphabet.append(f"##{letter}")

        alphabet.sort()        

        vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()

        self.splits = {
            word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
            for word in self.word_freqs.keys()
        }

        for i in range(vocab_size):
            scores = self.compute_pair_scores()
            best_pair = ""
            max_freq = None
            max_score= None
            for pair, score in scores.items():
                if max_score is None or max_score < score:
                    best_pair = pair
                    max_score = score
            self.splits = self.merge_pair(*best_pair)
            new_token = (
                best_pair[0] + best_pair[1][2:]
                if best_pair[1].startswith("##")
                else best_pair[0] + best_pair[1]
            )
            vocab.append(new_token)
            self.vocab = vocab
        return vocab
   
    def compute_pair_scores(self):
        letter_freqs = defaultdict(int)
        pair_freqs = defaultdict(int)
        for word, freq in self.word_freqs.items():
            split = self.splits[word]
            if len(split) == 1:
                letter_freqs[split[0]] += freq
                continue
            for i in range(len(split) - 1):
                pair = (split[i], split[i + 1])
                letter_freqs[split[i]] += freq
                pair_freqs[pair] += freq
            letter_freqs[split[-1]] += freq

        scores = {
            pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
            for pair, freq in pair_freqs.items()
        }
        return scores

    def merge_pair(self, a, b):
        for word in self.word_freqs:
            split = self.splits[word]
            if len(split) == 1:
                continue
            i = 0
            while i < len(split) - 1:
                if split[i] == a and split[i + 1] == b:
                    merge = a + b[2:] if b.startswith("##") else a + b
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            self.splits[word] = split
        return self.splits
    
    
    def encode_word(self, word):
        tokens = []
        while len(word) > 0:
            i = len(word)
            while i > 0 and word[:i] not in self.vocab:
                i -= 1
            if i == 0:
                return ["[UNK]"]
            tokens.append(word[:i])
            word = word[i:]
            if len(word) > 0:
                word = f"##{word}"
        return tokens
    
    
    def tokenize(self, text):
        # pre_tokenize_result = self.tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
        pre_tokenize_result = text.split()
        # pre_tokenized_text = [word for word, offset in pre_tokenize_result]
        pre_tokenized_text = [word for word in pre_tokenize_result]
        encoded_words = [self.encode_word(word) for word in pre_tokenized_text]
        return sum(encoded_words, [])


<h2> Train </h2>

In [ ]:
with open('All_Around_the_Moon.txt', encoding="utf8") as f:
    corpus = f.readlines()
vocab_size = 50

MyBPE = BPE(corpus=corpus, vocab_size=vocab_size)
BPE_Vocab =  MyBPE.train()

MyWordPiece = WordPiece(corpus=corpus, vocab_size=vocab_size)
WordPiece_Vocab =  MyWordPiece.train()

#BPE_Vocab, WordPiece_Vocab

<h2> Tokenization Text1 with BPE </h2>

In [ ]:
text1 = "This darkness is absolutely killing! If we ever take this trip again, it must be about the time of the sNew Moon!"
print(MyBPE.tokenize(text))

<h2> Tokenization Text1 with Wordpeice </h2>

In [ ]:
text1 = "This darkness is absolutely killing! If we ever take this trip again, it must be about the time of the sNew Moon!"
print(MyWordPiece.tokenize(text))

<h2> Tokenization Text2 with BPE </h2>

In [ ]:
text2 = "This is a tokenization task. Tokenization is the first step in a NLP pipeline. We will be comparing the tokens generated by each tokenization model"
print(MyBPE.tokenize(text2))

<h2> Tokenization Text2 with Wordpeice </h2>

In [ ]:
text2 = "This is a tokenization task. Tokenization is the first step in a NLP pipeline. We will be comparing the tokens generated by each tokenization model"
print(MyWordPiece.tokenize(text2))